# Lab3 - Assignment Sentiment

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook describes the LAB-3 assignment of the Text Mining course. It is about sentiment analysis.

The aims of the assignment are:
* Learn how to run a rule-based sentiment analysis module (VADER)
* Learn how to run a machine learning sentiment analysis module (Scikit-Learn/ Naive Bayes)
* Learn how to run scikit-learn metrics for the quantitative evaluation
* Learn how to perform and interpret a quantitative evaluation of the outcomes of the tools (in terms of Precision, Recall, and F<sub>1</sub>)
* Learn how to evaluate the results qualitatively (by examining the data) 
* Get insight into differences between the two applied methods
* Get insight into the effects of using linguistic preprocessing
* Be able to describe differences between the two methods in terms of their results
* Get insight into issues when applying these methods across different  domains

In this assignment, you are going to create your own gold standard set from 50 tweets. You will the VADER and scikit-learn classifiers to these tweets and evaluate the results by using evaluation metrics and inspecting the data.

We recommend you go through the notebooks in the following order:
* **Read the assignment (see below)**
* **Lab3.2-Sentiment-analysis-with-VADER.ipynb**
* **Lab3.3-Sentiment-analysis.with-scikit-learn.ipynb**
* **Answer the questions of the assignment (see below) using the provided notebooks and submit**

In this assignment you are asked to perform both quantitative evaluations and error analyses:
* a quantitative evaluation concerns the scores (Precision, Recall, and F<sub>1</sub>) provided by scikit's classification_report. It includes the scores per category, as well as micro and macro averages. Discuss whether the scores are balanced or not between the different categories (positive, negative, neutral) and between precision and recall. Discuss the shortcomings (if any) of the classifier based on these scores
* an error analysis regarding the misclassifications of the classifier. It involves going through the texts and trying to understand what has gone wrong. It servers to get insight in what could be done to improve the performance of the classifier. Do you observe patterns in misclassifications?  Discuss why these errors are made and propose ways to solve them.

## Credits
The notebooks in this block have been originally created by [Marten Postma](https://martenpostma.github.io) and [Isa Maks](https://research.vu.nl/en/persons/e-maks). Adaptations were made by [Filip Ilievski](http://ilievski.nl).

## Part I: VADER assignments


### Preparation (nothing to submit):
To be able to answer the VADER questions you need to know how the tool works. 
* Read more about the VADER tool in [this blog](http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html).  
* VADER provides 4 scores (positive, negative, neutral, compound). Be sure to understand what they mean and how they are calculated.
* VADER uses rules to handle linguistic phenomena such as negation and intensification. Be sure to understand which rules are used, how they work, and why they are important.
* VADER makes use of a sentiment lexicon. Have a look at the lexicon. Be sure to understand which information can be found there (lemma?, wordform?, part-of-speech?, polarity value?, word meaning?) What do all scores mean? https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt) 


### [3.5 points] Question1:

Regard the following sentences and their output as given by VADER. Regard sentences 1 to 7, and explain the outcome **for each sentence**. Take into account both the rules applied by VADER and the lexicon that is used. You will find that some of the results are reasonable, but others are not. Explain what is going wrong or not when correct and incorrect results are produced. 

```
INPUT SENTENCE 1 I love apples
VADER OUTPUT {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}
```
**answer:** VADER took every word seperately and evaluated the sentiment, it likely took the word 'love' and evaluated it as very positive, and the rest of the words as slightly positive/ generally neutral (neg score 0.0 implies it did not evaluate any word as negative), giving the whole sentence a compound score of around 0.64

```
INPUT SENTENCE 2 I don't love apples
VADER OUTPUT {'neg': 0.627, 'neu': 0.373, 'pos': 0.0, 'compound': -0.5216}
```
**answer:** VADER likely recognized 'don't' and 'love' together to evaluate it as quite negative, it's interesting to note though that it does recognize that sentence 2 is negative but less strongly than sentence 1 being positive, as sentence 1 is definetly strongly positive but sentence 2 is generally negative but 'don't love' does not imply a strong dislike.

```
INPUT SENTENCE 3 I love apples :-)
VADER OUTPUT {'neg': 0.0, 'neu': 0.133, 'pos': 0.867, 'compound': 0.7579}
```
**answer:** VADER likely evaluated the sentence so much more positive in comparison to sentence 1 due to the addition of the emoticon, since it is also able to handle and evaluate them
```
INPUT SENTENCE 4 These houses are ruins
VADER OUTPUT {'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'compound': -0.4404}
```
**answer:** I think VADER is quite poor at handling words with multiple meanings, like the word "ruins". Though the sentence is quite accuratly evaluated as neutral, it still leans quite close towards negative, since a house being a ruin could be a negative thing, it's well evaluated in my opinion.
```
INPUT SENTENCE 5 These houses are certainly not considered ruins
VADER OUTPUT {'neg': 0.0, 'neu': 0.51, 'pos': 0.49, 'compound': 0.5867}
```
**answer:** This is one of those sentences where the evaluation doesn't seem to make much sense. VADER evaluated words it considers very positive like 'certainly' well and took the rest of the sentence as neutral, but it likely tried negating 'not (negative) considered ruins(negative)' resulting in the sentence having a positive compound evaluation
```
INPUT SENTENCE 6 He lies in the chair in the garden
VADER OUTPUT {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.4215}
```
**answer:** Although this definitely seems like a neutral sentence, it's really likely the word "lies" is considered negative in VADER's lexicon, probably as the human evaluators thought of the word lies as in lying, not lying down, resulting in a negative compound score.
```
INPUT SENTENCE 7 This house is like any house
VADER OUTPUT {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.3612}
```
**answer:** Although the neutral compound score is definetly accurate, it's possible the slight positive bias comes from the word 'like'
```

### [Points: 2.5] Exercise 2: Collecting 50 tweets for evaluation
Collect 50 tweets. Try to find tweets that are interesting for sentiment analysis, e.g., very positive, neutral, and negative tweets. These could be your own tweets (typed in) or collected from the Twitter stream. If you have trouble accessing Twitter, try to find an existing dataset (on websites like kaggle or huggingface).

We will store the tweets in the file **my_tweets.json** (use a text editor to edit).
For each tweet, you should insert:
* sentiment analysis label: negative | neutral | positive (this you determine yourself, this is not done by a computer)
* the text of the tweet
* the Tweet-URL

from:
```
    "1": {
        "sentiment_label": "",
        "text_of_tweet": "",
        "tweet_url": "",
```
to:
```
"1": {
        "sentiment_label": "positive",
        "text_of_tweet": "All across America people chose to get involved, get engaged and stand up. Each of us can make a difference, and all of us ought to try. So go keep changing the world in 2018.",
        "tweet_url" : "https://twitter.com/BarackObama/status/946775615893655552",
    },
```

You can load your tweets with human annotation in the following way.

In [23]:
import json

In [24]:
my_tweets = json.load(open('my_tweets.json'))

In [25]:
for id_, tweet_info in my_tweets.items():
    print(id_, tweet_info)
    break

1 {'sentiment_label': 'positive', 'text_of_tweet': 'oh that resist choreo is READYYYYY', 'tweet_url': 'https://x.com/tyuntete/status/1895414524733481051'}


### [5 points] Question 3:

Run VADER on your own tweets (see function **run_vader** from notebook **Lab2-Sentiment-analysis-using-VADER.ipynb**). You can use the code snippet below this explanation as a starting point. 
* [2.5 points] a. Perform a quantitative evaluation. Explain the different scores, and explain which scores are most relevant and why.
* [2.5 points] b. Perform an error analysis: select 10 positive, 10 negative and 10 neutral tweets that are not correctly classified and try to understand why. Refer to the VADER-rules and the VADER-lexicon. Of course, if there are less than 10 errors for a category, you only have to check those. For example, if there are only 5 errors for positive tweets, you just describe those.

In [26]:
def vader_output_to_label(vader_output):
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    elif compound > 0.0:
        return 'positive'
    
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.0}) == 'neutral'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.01}) == 'positive'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': -0.01}) == 'negative'

In [27]:
tweets = []
all_vader_output = []
gold = []

# settings (to change for different experiments)
to_lemmatize = True 
pos = set()

for id_, tweet_info in my_tweets.items():
    the_tweet = tweet_info['text_of_tweet']
    vader_output = ''# run vader
    vader_label = ''# convert vader output to category
    
    tweets.append(the_tweet)
    all_vader_output.append(vader_label)
    gold.append(tweet_info['sentiment_label'])
    
# use scikit-learn's classification report

### [4 points] Question 4:
Run VADER on the set of airline tweets with the following settings:

* Run VADER (as it is) on the set of airline tweets 
* Run VADER on the set of airline tweets after having lemmatized the text
* Run VADER on the set of airline tweets with only adjectives
* Run VADER on the set of airline tweets with only adjectives and after having lemmatized the text
* Run VADER on the set of airline tweets with only nouns
* Run VADER on the set of airline tweets with only nouns and after having lemmatized the text
* Run VADER on the set of airline tweets with only verbs
* Run VADER on the set of airline tweets with only verbs and after having lemmatized the text

* [1 point] a. Generate for all separate experiments the classification report, i.e., Precision, Recall, and F<sub>1</sub> scores per category as well as micro and macro averages. **Use a different code cell (or multiple code cells) for each experiment.**
* [3 points] b. Compare the scores and explain what they tell you.
* - Does lemmatisation help? Explain why or why not.
* - Are all parts of speech equally important for sentiment analysis? Explain why or why not.

In [28]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Users\Tammy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [42]:
import pandas as pd
import nltk
import pathlib
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.metrics import classification_report
import string
import os

nltk.download("vader_lexicon")
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("averaged_perceptron_tagger")
nltk.download('averaged_perceptron_tagger_eng')
nltk.download("wordnet")
nltk.download("stopwords")

sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))


airline_tweets_folder = "C:/Users/Tammy/OneDrive - John Abbott College/YEAR 1/Documents/GitHub/ba-text-mining-gr43/lab_sessions/lab3/airlinetweets/airlinetweets"
categories = ["negative", "neutral", "positive"]
tweets_data = []


for category in categories:
    folder_path = f"{airline_tweets_folder}/{category}"
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, encoding="utf-8") as file:
                    tweet_text = file.read().strip()
                    tweets_data.append((tweet_text, category))



def get_sentiment_score(tweet):
    return sia.polarity_scores(tweet)

def get_label(score):
    if score['compound'] >= 0.05:
        return 'positive'
    elif score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'


def print_classification_report(scores, label):
    predictions = [] 
    true_labels = [] 
    
    for tweet, category, score in scores:
        if score['compound'] >= 0.05:
            predictions.append('positive')  
        elif score['compound'] <= -0.05:
            predictions.append('negative')
        else:
            predictions.append('neutral')  
        
        
        true_labels.append(category)
    
    
    if not true_labels or not predictions:
        print(f"Error: true_labels or predictions are empty for {label} tweets.")
        print(f"true_labels: {true_labels}")
        print(f"predictions: {predictions}")
        return

    print(f"Classification Report for {label} Tweets:")
    print(classification_report(true_labels, predictions))


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Tammy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tammy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Tammy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tammy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Tammy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tammy\AppData\Roam

In [43]:
# VADER on airline tweets
raw_scores = []

for tweet, category in tweets_data:
    score = get_sentiment_score(tweet)
    
    raw_scores.append((tweet, category, score))
print_classification_report(raw_scores, "Raw")

Classification Report for Raw Tweets:
              precision    recall  f1-score   support

    negative       0.80      0.49      0.61      1750
     neutral       0.58      0.53      0.55      1515
    positive       0.57      0.87      0.69      1490

    accuracy                           0.62      4755
   macro avg       0.65      0.63      0.62      4755
weighted avg       0.66      0.62      0.62      4755



In [44]:
# Running VADER on lemmatized tweets
def lemmatize_text(tweet):
    words = tweet.split()
    
    lemmatized_words = []
    for word in words:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    
    return ' '.join(lemmatized_words)

lemmatized_tweets_data = []

for tweet, category in tweets_data:
    lemmatized_tweet = lemmatize_text(tweet)
    lemmatized_tweets_data.append((lemmatized_tweet, category))

lemmatized_scores = []

for tweet, category in lemmatized_tweets_data:
    score = get_sentiment_score(tweet)
    lemmatized_scores.append((tweet, category, score))

print_classification_report(lemmatized_scores, "Lemmatized")

Classification Report for Lemmatized Tweets:
              precision    recall  f1-score   support

    negative       0.79      0.49      0.60      1750
     neutral       0.58      0.52      0.55      1515
    positive       0.56      0.87      0.68      1490

    accuracy                           0.62      4755
   macro avg       0.64      0.63      0.61      4755
weighted avg       0.65      0.62      0.61      4755



In [45]:
# VADER on only adjectives
def extract_pos(tweet, pos_tags):
    words = word_tokenize(tweet)
    tagged = pos_tag(words)   
    filtered_words = []
    for word, tag in tagged:
        if tag in pos_tags:
            filtered_words.append(word)    
    return ' '.join(filtered_words)

adjective_tweets = []

for tweet, category in tweets_data:
    adjectives_only_tweet = extract_pos(tweet, {'JJ', 'JJR', 'JJS'})
    adjective_tweets.append((adjectives_only_tweet, category))

adjective_scores = []

for tweet, category in adjective_tweets:
    score = get_sentiment_score(tweet)
    adjective_scores.append((tweet, category, score))

print_classification_report(adjective_scores, "Adjective")

Classification Report for Adjective Tweets:
              precision    recall  f1-score   support

    negative       0.83      0.15      0.26      1750
     neutral       0.49      0.78      0.60      1515
    positive       0.35      0.47      0.40      1490

    accuracy                           0.45      4755
   macro avg       0.55      0.47      0.42      4755
weighted avg       0.57      0.45      0.41      4755



In [46]:
# VADER on only adjectives, lemmatized 
lemmatized_adjective_tweets = []

for tweet, category in adjective_tweets:
    lemmatized_tweet = lemmatize_text(tweet)
    lemmatized_adjective_tweets.append((lemmatized_tweet, category))

lemmatized_adjective_scores = []

for tweet, category in lemmatized_adjective_tweets:
    score = get_sentiment_score(tweet) 
    lemmatized_adjective_scores.append((tweet, category, score))

print_classification_report(lemmatized_adjective_scores, "Lemmatized Adjective")

Classification Report for Lemmatized Adjective Tweets:
              precision    recall  f1-score   support

    negative       0.83      0.15      0.26      1750
     neutral       0.49      0.78      0.60      1515
    positive       0.35      0.47      0.40      1490

    accuracy                           0.45      4755
   macro avg       0.55      0.47      0.42      4755
weighted avg       0.57      0.45      0.41      4755



In [47]:
# VADER on only nouns 
noun_tweets = []

for tweet, category in tweets_data:
    noun_tweet = extract_pos(tweet, {'NN', 'NNS', 'NNP', 'NNPS'})
    noun_tweets.append((noun_tweet, category))

noun_scores = []

for tweet, category in noun_tweets:
    score = get_sentiment_score(tweet)
    noun_scores.append((tweet, category, score))

print_classification_report(noun_scores, "Noun")

Classification Report for Noun Tweets:
              precision    recall  f1-score   support

    negative       0.77      0.22      0.35      1750
     neutral       0.42      0.71      0.52      1515
    positive       0.55      0.62      0.58      1490

    accuracy                           0.50      4755
   macro avg       0.58      0.52      0.48      4755
weighted avg       0.59      0.50      0.48      4755



In [48]:
# VADER on only nouns, lemmatized 
lemmatized_noun_tweets = []
for tweet, category in noun_tweets:
    lemmatized_tweet = lemmatize_text(tweet)
    lemmatized_noun_tweets.append((lemmatized_tweet, category))

lemmatized_noun_scores = []

for tweet, category in lemmatized_noun_tweets:
    score = get_sentiment_score(tweet)
    lemmatized_noun_scores.append((tweet, category, score))

print_classification_report(lemmatized_noun_scores, "Lemmatized Noun")

Classification Report for Lemmatized Noun Tweets:
              precision    recall  f1-score   support

    negative       0.77      0.23      0.35      1750
     neutral       0.42      0.70      0.52      1515
    positive       0.55      0.62      0.58      1490

    accuracy                           0.50      4755
   macro avg       0.58      0.52      0.49      4755
weighted avg       0.59      0.50      0.48      4755



In [49]:
# VADER on only verbs

verb_tweets = []
for tweet, category in tweets_data:
    verb_tweet = extract_pos(tweet, {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'})
    verb_tweets.append((verb_tweet, category))

verb_scores = []

for tweet, category in verb_tweets:
    score = get_sentiment_score(tweet)
    verb_scores.append((tweet, category, score))

print_classification_report(verb_scores, "Verb")

Classification Report for Verb Tweets:
              precision    recall  f1-score   support

    negative       0.74      0.19      0.30      1750
     neutral       0.38      0.78      0.51      1515
    positive       0.30      0.24      0.27      1490

    accuracy                           0.39      4755
   macro avg       0.47      0.40      0.36      4755
weighted avg       0.49      0.39      0.36      4755



In [50]:
# VADER on only verbs, lemmatized 
lemmatized_verb_tweets = []

for tweet, category in verb_tweets:
    lemmatized_tweet = lemmatize_text(tweet)
    lemmatized_verb_tweets.append((lemmatized_tweet, category))

lemmatized_verb_scores = []

for tweet, category in lemmatized_verb_tweets:
    score = get_sentiment_score(tweet)
    lemmatized_verb_scores.append((tweet, category, score))

print_classification_report(lemmatized_verb_scores, "Lemmatized Verb")

Classification Report for Lemmatized Verb Tweets:
              precision    recall  f1-score   support

    negative       0.73      0.18      0.29      1750
     neutral       0.38      0.77      0.51      1515
    positive       0.30      0.25      0.27      1490

    accuracy                           0.39      4755
   macro avg       0.47      0.40      0.36      4755
weighted avg       0.48      0.39      0.35      4755



b. Comparing the scores - answers

1. Looking at the scores and comparing them overall, we can see that adjectives have higher scores for all categories compared to verbs and nouns, which tells us that sentiment is expressed the best with adjectives. If comparing nouns and verbs, nouns have slightly higher macro and weighted averages for all categories (precision, recall and f1 score), only the weighted average for recall is the exact same (0.54).
2. In theory, lemmatisation does help - it simplifies the words by transforming it to its root form, thus it reduces variation and vocabulary becomes less complicated. The only downside could be that some words might lose their actual precise meaning if the prefix or sufix is removed. However, in this case above, the scores for raw and lemmatized tweets are the same; lemmatization does not impact the scores at all.
3. Not all parts of speech equally important for sentiment analysis - the most important are adjectives, then nouns and verbs which add context as well, but for instance pronouns or prepositions do not contribute to sentiment analysis (words like "in", "at", "she" do not express emotion at all, while "wonderful" and "joy" do.

## Part II: scikit-learn assignments
### [4 points] Question 5
Train the scikit-learn classifier (Naive Bayes) using the airline tweets.

+ Train the model on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=2)
+ Train with different settings:
    + with respect to vectorizing: TF-IDF ('airline_tfidf') vs. Bag of words representation ('airline_count') 
    + with respect to the frequency threshold (min_df). Carry out experiments with increasing values for document frequency (min_df = 2; min_df = 5; min_df =10) 
* [1 point] a. Generate a classification_report for all experiments
* [3 points] b. Look at the results of the experiments with the different settings and try to explain why they differ: 
    + which category performs best, is this the case for any setting?
    + does the frequency threshold affect the scores? Why or why not according to you?

In [27]:
import os
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

data_path = "C:/Users/lisam/OneDrive/Year 3/Period 4/Text Mining/ba-text-mining-gr43/lab_sessions/lab3/airlinetweets"

def load_data(data_path):
    texts, labels = [], []
    for sentiment in ['negative', 'neutral', 'positive']:
        folder_path = os.path.join(data_path, sentiment)
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                with open(file_path, "r", encoding="utf-8") as f:
                    texts.append(f.read().strip())
                    labels.append(sentiment)
    return texts, labels

texts, labels = load_data(data_path)

map_labels = {'negative': 0, 'neutral': 1, 'positive': 2}
y = np.array([map_labels[label] for label in labels])

X_train, X_test, y_train, y_test = train_test_split(texts, y, test_size=0.2, random_state=42, stratify=y)

vectorizers = {
    "TF-IDF (min_df=2)": TfidfVectorizer(min_df=2),
    "TF-IDF (min_df=5)": TfidfVectorizer(min_df=5),
    "TF-IDF (min_df=10)": TfidfVectorizer(min_df=10),
    "Bag of Words (min_df=2)": CountVectorizer(min_df=2),
    "Bag of Words (min_df=5)": CountVectorizer(min_df=5),
    "Bag of Words (min_df=10)": CountVectorizer(min_df=10),
}

results = {}

for name, vectorizer in vectorizers.items():
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    model = svm.LinearSVC()
    model.fit(X_train_vec, y_train)
    
    y_pred = model.predict(X_test_vec)
    
    report = classification_report(y_test, y_pred, target_names=map_labels.keys())
    results[name] = report
    print(f"Classification Report for {name}:\n", report)
    print("-"*80)


Classification Report for TF-IDF (min_df=2):
               precision    recall  f1-score   support

    negative       0.80      0.92      0.85       350
     neutral       0.79      0.73      0.76       303
    positive       0.88      0.80      0.84       298

    accuracy                           0.82       951
   macro avg       0.82      0.81      0.82       951
weighted avg       0.82      0.82      0.82       951

--------------------------------------------------------------------------------
Classification Report for TF-IDF (min_df=5):
               precision    recall  f1-score   support

    negative       0.79      0.92      0.85       350
     neutral       0.79      0.73      0.76       303
    positive       0.89      0.79      0.83       298

    accuracy                           0.82       951
   macro avg       0.82      0.81      0.81       951
weighted avg       0.82      0.82      0.82       951

-----------------------------------------------------------------

### [4 points] Question 6: Inspecting the best scoring features 

+ Train the scikit-learn classifier (Naive Bayes) model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
* [1 point] a. Generate the list of best scoring features per class (see function **important_features_per_class** below) [1 point]
* [3 points] b. Look at the lists and consider the following issues: 
    + [1 point] Which features did you expect for each separate class and why?
    + [1 point] Which features did you not expect and why ? 
    + [1 point] The list contains all kinds of words such as names of airlines, punctuation, numbers and content words (e.g., 'delay' and 'bad'). Which words would you remove or keep when trying to improve the model and why? 

In [ ]:
def important_features_per_class(vectorizer,classifier,n=80):
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:
#important_features_per_class(airline_vec, clf)

### [Optional! (will not  be graded)] Question 7
Train the model on airline tweets and test it on your own set of tweets
+ Train the model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
+ Apply the model on your own set of tweets and generate the classification report
* [1 point] a. Carry out a quantitative analysis.
* [1 point] b. Carry out an error analysis on 10 correctly and 10 incorrectly classified tweets and discuss them
* [2 points] c. Compare the results (cf. classification report) with the results obtained by VADER on the same tweets and discuss the differences.

### [Optional! (will not be graded)] Question 8: trying to improve the model
* [2 points] a. Think of some ways to improve the scikit-learn Naive Bayes model by playing with the settings or applying linguistic preprocessing (e.g., by filtering on part-of-speech, or removing punctuation). Do not change the classifier but continue using the Naive Bayes classifier. Explain what the effects might be of these other settings 
+ [1 point] b. Apply the model with at least one new setting (train on the airline tweets using 80% training, 20% test) and generate the scores
* [1 point] c. Discuss whether the model achieved what you expected.

## End of this notebook